
# MIMIC-III EDA and prep

Cleaning and prep of diagnostic notes and related data from MIMIC-III dataset

Repo: <a href="https://github.com/mvmagni/mimic-iii_LL">github.com/mvmagni/mimic-iii_LL</a>

Dataset location:

    wget -r -N -c -np --user [username] --ask-password https://physionet.org/files/mimiciii/1.4/

## Steps

1. Import data
    * NOTEEVENTS.csv
    * DIAGNOSIS_ICD.csv
2. DIAGNOSIS_ICD prep
    * Action: Take first record over HADM_ID where SEQ_NUM==1 which contains the primary diagnoses. 
    * Data check: SEQ_NUM == 1 for all records, no nulls
    * Data check: HADM_ID should be unique across all records, no nulls
3. NOTEEVENTS prep
    * Action: Trim to only include HADM_ID and TEXT columns
    * Action: Drop null rows 231,836 NULL values in HADM_ID
    * Data check: TEXT has no null values
    * Data check: HADM_ID has no null values
4. Combine TEXT in NOTEEVENTS for all HADM_ID
    * Action: create new dataframe with unique HADM_ID where all TEXT values combined
    * Data check: ensure combined frame has higher text length values (e.g. mean, max, stdev)
    * Data check: combined TEXT has no null values
    * Data check: unique HADM_ID across all records
    * Data check: HADM_ID has no null values
5. Output 1: Create dataframe where TEXT is not joined but has ICD9 Code for SEQ==1
    * Data check: check that HADM_ID values all exist in both NOTES and DIAGNOSES before merge
    * Action: Merge NOTES and DIAGNOSES (with uncombined TEXT) 
    * Data check: only common HADM_ID should remain after merge
    * Data check: TEXT has no null values
    * Data check: ICD9_CODE has no null values
    * Data check: SEQ_NUM == 1 for all records 
    * Persist dataframe
6. Output 2: Create dataframe where TEXT are joined and has ICD9 Code for SEQ==1
    * Data check: check that HADM_ID values all exist in both NOTES and DIAGNOSES before merge
    * Action: Merge NOTES and DIAGNOSES (with uncombined TEXT) 
    * Data check: only common HADM_ID should remain after merge
    * Data check: TEXT has no null values
    * Data check: ICD9_CODE has no null values
    * Data check: SEQ_NUM == 1 for all records 
    * Persist dataframe
7. Explore ICD9_CODE frequency in combined and separate NOTES data sets
8. Persist custom set of stopwords for later use


# Initialize environment    

In [1]:
import sys
import os

WORKING_DIR = f'{os.getcwd()}'

# Magritte has the utility functions we will be using
# Set MAGRITTE_DIR to where you checked out the github repo
# URL: github.com/mvmagni/magritte
MAGRITTE_DIR = f'{WORKING_DIR}/../../magritte'
UTILITIES_DIR = f'{MAGRITTE_DIR}/utilities'

# Directory for loading and storing files
DATA_DIR = f'{WORKING_DIR}/../../data/mimiciii'

# Add the UTILITY_DIR to the path to import files
sys.path.append(UTILITIES_DIR)

print(f'Environment configured:')
print(f'WORKING_DIR:   {WORKING_DIR}')
print(f'MAGRITTE_DIR:  {MAGRITTE_DIR}')
print(f'UTILITIES_DIR: {UTILITIES_DIR}')
print(f'DATA_DIR:      {DATA_DIR}')


Environment configured:
WORKING_DIR:   /home/magni/ML_Root/project_root/mimic-iii_LL/notebooks
MAGRITTE_DIR:  /home/magni/ML_Root/project_root/mimic-iii_LL/notebooks/../../magritte
UTILITIES_DIR: /home/magni/ML_Root/project_root/mimic-iii_LL/notebooks/../../magritte/utilities
DATA_DIR:      /home/magni/ML_Root/project_root/mimic-iii_LL/notebooks/../../data/mimiciii


In [2]:
import pandas as pd
import DataUtils
import pickle

# Load Data (MIMIC-III Dataset)

In [3]:
%%time
# Loading two tables from MIMIC-III
# 1) DIAGNOSES_ICD.csv.gz
# 2) NOTEEVENTS.csv.gz

DIAGNOSES_ICD_DF = pd.read_csv(f'{DATA_DIR}/DIAGNOSES_ICD.csv.gz',
                              compression='gzip'
                             )

DATE_COLS=['CHARTTIME','STORETIME', 'CHARTDATE']
NOTES_DF = pd.read_csv(f'{DATA_DIR}/NOTEEVENTS.csv.gz',
                      parse_dates=DATE_COLS,
                      compression='gzip'
                     )

CPU times: user 30.7 s, sys: 1.37 s, total: 32.1 s
Wall time: 33.7 s


## DIAGNOSES_ICD summary

In [4]:
DataUtils.exploreDataframe(DIAGNOSES_ICD_DF)

dataframe shape: (651047, 5)

dataframe info: 
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 651047 entries, 0 to 651046
Data columns (total 5 columns):
 #   Column      Non-Null Count   Dtype  
---  ------      --------------   -----  
 0   ROW_ID      651047 non-null  int64  
 1   SUBJECT_ID  651047 non-null  int64  
 2   HADM_ID     651047 non-null  int64  
 3   SEQ_NUM     651000 non-null  float64
 4   ICD9_CODE   651000 non-null  object 
dtypes: float64(1), int64(3), object(1)
memory usage: 24.8+ MB
None

Null value count by column:


ROW_ID         0
SUBJECT_ID     0
HADM_ID        0
SEQ_NUM       47
ICD9_CODE     47
dtype: int64



First 5 in dataframe


,ROW_ID,SUBJECT_ID,HADM_ID,SEQ_NUM,ICD9_CODE
0,1297,109,172335,1.0,40301
1,1298,109,172335,2.0,486
2,1299,109,172335,3.0,58281
3,1300,109,172335,4.0,5855
4,1301,109,172335,5.0,4254



Last 5 in dataframe


,ROW_ID,SUBJECT_ID,HADM_ID,SEQ_NUM,ICD9_CODE
651042,639798,97503,188195,2.0,20280
651043,639799,97503,188195,3.0,V5869
651044,639800,97503,188195,4.0,V1279
651045,639801,97503,188195,5.0,5275
651046,639802,97503,188195,6.0,5569


In [5]:
_, _ = DataUtils.showUniqueColVals(DIAGNOSES_ICD_DF, 'ICD9_CODE')

Data type of column [ICD9_CODE] is: object
Total number of rows: 651047
Unique values in column: 6985 [percent unique: 1.0999999999999999%]
Null values in column: 47
List of unique values:
['40301' '486' '58281' ... 'E0070' '6940' '20930']

Top 5 records by frequency for ICD9_CODE
     ICD9_CODE  record_count
1962      4019         20703
2109      4280         13111
2098     42731         12891
2019     41401         12429
2957      5849          9119

Bottom 5 records by frequency for ICD9_CODE
     ICD9_CODE  record_count
6983     V9103             1
6854      V562             1
1336      3060             1
202      07953             1
1338      3062             1


## NOTEEVENTS summary

In [6]:
DataUtils.exploreDataframe(NOTES_DF)

dataframe shape: (2083180, 11)

dataframe info: 
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2083180 entries, 0 to 2083179
Data columns (total 11 columns):
 #   Column       Dtype         
---  ------       -----         
 0   ROW_ID       int64         
 1   SUBJECT_ID   int64         
 2   HADM_ID      float64       
 3   CHARTDATE    datetime64[ns]
 4   CHARTTIME    datetime64[ns]
 5   STORETIME    datetime64[ns]
 6   CATEGORY     object        
 7   DESCRIPTION  object        
 8   CGID         float64       
 9   ISERROR      float64       
 10  TEXT         object        
dtypes: datetime64[ns](3), float64(3), int64(2), object(3)
memory usage: 174.8+ MB
None

Null value count by column:


ROW_ID               0
SUBJECT_ID           0
HADM_ID         231836
CHARTDATE            0
CHARTTIME       316566
STORETIME       836776
CATEGORY             0
DESCRIPTION          0
CGID            836776
ISERROR        2082294
TEXT                 0
dtype: int64



First 5 in dataframe


,ROW_ID,SUBJECT_ID,HADM_ID,CHARTDATE,CHARTTIME,STORETIME,CATEGORY,DESCRIPTION,CGID,ISERROR,TEXT
0,174,22532,167853.0,2151-08-04,NaT,NaT,Discharge summary,Report,NaN,NaN,Admission Date: [**2151-7-16**] Dischar...
1,175,13702,107527.0,2118-06-14,NaT,NaT,Discharge summary,Report,NaN,NaN,Admission Date: [**2118-6-2**] Discharg...
2,176,13702,167118.0,2119-05-25,NaT,NaT,Discharge summary,Report,NaN,NaN,Admission Date: [**2119-5-4**] D...
3,177,13702,196489.0,2124-08-18,NaT,NaT,Discharge summary,Report,NaN,NaN,Admission Date: [**2124-7-21**] ...
4,178,26880,135453.0,2162-03-25,NaT,NaT,Discharge summary,Report,NaN,NaN,Admission Date: [**2162-3-3**] D...



Last 5 in dataframe


,ROW_ID,SUBJECT_ID,HADM_ID,CHARTDATE,CHARTTIME,STORETIME,CATEGORY,DESCRIPTION,CGID,ISERROR,TEXT
2083175,2070657,31097,115637.0,2132-01-21,2132-01-21 03:27:00,2132-01-21 03:38:00,Nursing/other,Report,17581.0,NaN,NPN\n\n\n#1 Infant remains in RA with O2 sats...
2083176,2070658,31097,115637.0,2132-01-21,2132-01-21 09:50:00,2132-01-21 09:53:00,Nursing/other,Report,19211.0,NaN,"Neonatology\nDOL #5, CGA 36 weeks.\n\nCVR: Con..."
2083177,2070659,31097,115637.0,2132-01-21,2132-01-21 16:42:00,2132-01-21 16:44:00,Nursing/other,Report,20104.0,NaN,Family Meeting Note\nFamily meeting held with ...
2083178,2070660,31097,115637.0,2132-01-21,2132-01-21 18:05:00,2132-01-21 18:16:00,Nursing/other,Report,16023.0,NaN,NPN 1800\n\n\n#1 Resp: [**Known lastname 2243*...
2083179,2070661,31097,115637.0,2132-01-21,2132-01-21 18:05:00,2132-01-21 18:31:00,Nursing/other,Report,16023.0,NaN,NPN 1800\nNursing Addendum:\n[**Known lastname...


In [7]:
_ = DataUtils.showUniqueColVals(NOTES_DF, 'HADM_ID')

Data type of column [HADM_ID] is: float64
Total number of rows: 2083180
Unique values in column: 58362 [percent unique: 2.8000000000000003%]
Null values in column: 231836
List of unique values:
[167853. 107527. 167118. ... 132855. 115098. 175166.]

Top 5 records by frequency for HADM_ID
        HADM_ID  record_count
10266  117448.0          1233
23848  140792.0          1144
22421  138363.0          1099
44071  175448.0          1084
53982  192431.0          1073

Bottom 5 records by frequency for HADM_ID
        HADM_ID  record_count
29180  149973.0             1
56293  196399.0             1
38232  165601.0             1
38166  165500.0             1
38108  165375.0             1


In [8]:
_ = DataUtils.showUniqueColVals(NOTES_DF, 'CATEGORY', showRecords=15)

Data type of column [CATEGORY] is: object
Total number of rows: 2083180
Unique values in column: 15 [percent unique: 0.0%]
Null values in column: 0
List of unique values:
['Discharge summary' 'Echo' 'ECG' 'Nursing' 'Physician ' 'Rehab Services'
 'Case Management ' 'Respiratory ' 'Nutrition' 'General' 'Social Work'
 'Pharmacy' 'Consult' 'Radiology' 'Nursing/other']

Top 15 records by frequency for CATEGORY
             CATEGORY  record_count
7       Nursing/other        822497
11          Radiology        522279
6             Nursing        223556
3                 ECG        209051
10         Physician         141624
2   Discharge summary         59652
4                Echo         45794
13       Respiratory          31739
8           Nutrition          9418
5             General          8301
12     Rehab Services          5431
14        Social Work          2670
0    Case Management            967
9            Pharmacy           103
1             Consult            98

Bottom 15 reco


# DIAGNOSES_ICD explore and prep

In [9]:
DataUtils.exploreDataframe(DIAGNOSES_ICD_DF)

dataframe shape: (651047, 5)

dataframe info: 
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 651047 entries, 0 to 651046
Data columns (total 5 columns):
 #   Column      Non-Null Count   Dtype  
---  ------      --------------   -----  
 0   ROW_ID      651047 non-null  int64  
 1   SUBJECT_ID  651047 non-null  int64  
 2   HADM_ID     651047 non-null  int64  
 3   SEQ_NUM     651000 non-null  float64
 4   ICD9_CODE   651000 non-null  object 
dtypes: float64(1), int64(3), object(1)
memory usage: 24.8+ MB
None

Null value count by column:


ROW_ID         0
SUBJECT_ID     0
HADM_ID        0
SEQ_NUM       47
ICD9_CODE     47
dtype: int64



First 5 in dataframe


,ROW_ID,SUBJECT_ID,HADM_ID,SEQ_NUM,ICD9_CODE
0,1297,109,172335,1.0,40301
1,1298,109,172335,2.0,486
2,1299,109,172335,3.0,58281
3,1300,109,172335,4.0,5855
4,1301,109,172335,5.0,4254



Last 5 in dataframe


,ROW_ID,SUBJECT_ID,HADM_ID,SEQ_NUM,ICD9_CODE
651042,639798,97503,188195,2.0,20280
651043,639799,97503,188195,3.0,V5869
651044,639800,97503,188195,4.0,V1279
651045,639801,97503,188195,5.0,5275
651046,639802,97503,188195,6.0,5569


In [10]:
# Choose only the first record
# Business process: First record contains primary diagnosis for the patient

# Original research project filtered on keep=first. Used SEQ_NUM==1.0
#DIAGNOSES_ICD_DF_working = DIAGNOSES_ICD_DF.sort_values('SEQ_NUM').drop_duplicates('HADM_ID', keep='first')
DIAGNOSES_ICD_DF_working = DIAGNOSES_ICD_DF[(DIAGNOSES_ICD_DF['SEQ_NUM'] == 1.0)]

# Check value of SEQ_NUM. Should all be 1.0
_ = DataUtils.showUniqueColVals(DIAGNOSES_ICD_DF_working, 'SEQ_NUM', showRecords=10)

Data type of column [SEQ_NUM] is: float64
Total number of rows: 58929
Unique values in column: 1 [percent unique: 0.0%]
Null values in column: 0
List of unique values:
[1.]

Top 10 records by frequency for SEQ_NUM
   SEQ_NUM  record_count
0      1.0         58929

Bottom 10 records by frequency for SEQ_NUM
   SEQ_NUM  record_count
0      1.0         58929


In [11]:
# Check counts for ICD9_CODE
_ = DataUtils.showUniqueColVals(DIAGNOSES_ICD_DF_working, 
                                'ICD9_CODE', showRecords=5)

Data type of column [ICD9_CODE] is: object
Total number of rows: 58929
Unique values in column: 2789 [percent unique: 4.7%]
Null values in column: 0
List of unique values:
['40301' '53100' '1915' ... '5730' '2592' '0529']

Top 5 records by frequency for ICD9_CODE
     ICD9_CODE  record_count
2749     V3000          3566
837      41401          3498
2750     V3001          2757
47        0389          2069
826      41071          1751

Bottom 5 records by frequency for ICD9_CODE
     ICD9_CODE  record_count
2788     V7281             1
622      29289             1
625      29522             1
626      29524             1
627      29590             1


In [12]:
# Check uniqueness of HADM_ID, should be 100% unique
_ = DataUtils.showUniqueColVals(DIAGNOSES_ICD_DF_working, 'HADM_ID', showRecords=1)

Data type of column [HADM_ID] is: int64
Total number of rows: 58929
Unique values in column: 58929 [percent unique: 100.0%]
Null values in column: 0
List of unique values:
[172335 173633 174105 ... 189314 168949 188195]

Top 1 records by frequency for HADM_ID
   HADM_ID  record_count
0   100001             1

Bottom 1 records by frequency for HADM_ID
   HADM_ID  record_count
0   100001             1


In [13]:
# Sanity check: DIAGNOSES_ICD_DF_working

# 1) SEQ_NUM==1 for all records
assert(DIAGNOSES_ICD_DF_working['SEQ_NUM'].loc[DIAGNOSES_ICD_DF['SEQ_NUM'].isin([1.0])].nunique())==1
print(f'PASS: SEQ_NUM is only 1.0 for all entries')

# 2) SEQ_NUM has no nulls
assert(DIAGNOSES_ICD_DF_working['SEQ_NUM'].isnull().sum()==0)
print(f'PASS: SEQ_NUM contains no null values')

# 3) HADM_ID == 100% unique
assert(DIAGNOSES_ICD_DF_working['HADM_ID'].nunique()==len(DIAGNOSES_ICD_DF_working))
print(f'PASS: HADM_ID is 100% unique')

# 4) HADM_ID has no nulls
assert(DIAGNOSES_ICD_DF_working['HADM_ID'].isnull().sum()==0)
print(f'PASS: HADM_ID contains no null values')



PASS: SEQ_NUM is only 1.0 for all entries
PASS: SEQ_NUM contains no null values
PASS: HADM_ID is 100% unique
PASS: HADM_ID contains no null values



# NOTEEVENTS explore and prep

In [14]:
# Explore data frames for cleaning and prep
DataUtils.exploreDataframe(NOTES_DF, showRecords=1)

dataframe shape: (2083180, 11)

dataframe info: 
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2083180 entries, 0 to 2083179
Data columns (total 11 columns):
 #   Column       Dtype         
---  ------       -----         
 0   ROW_ID       int64         
 1   SUBJECT_ID   int64         
 2   HADM_ID      float64       
 3   CHARTDATE    datetime64[ns]
 4   CHARTTIME    datetime64[ns]
 5   STORETIME    datetime64[ns]
 6   CATEGORY     object        
 7   DESCRIPTION  object        
 8   CGID         float64       
 9   ISERROR      float64       
 10  TEXT         object        
dtypes: datetime64[ns](3), float64(3), int64(2), object(3)
memory usage: 174.8+ MB
None

Null value count by column:


ROW_ID               0
SUBJECT_ID           0
HADM_ID         231836
CHARTDATE            0
CHARTTIME       316566
STORETIME       836776
CATEGORY             0
DESCRIPTION          0
CGID            836776
ISERROR        2082294
TEXT                 0
dtype: int64



First 1 in dataframe


,ROW_ID,SUBJECT_ID,HADM_ID,CHARTDATE,CHARTTIME,STORETIME,CATEGORY,DESCRIPTION,CGID,ISERROR,TEXT
0,174,22532,167853.0,2151-08-04,NaT,NaT,Discharge summary,Report,NaN,NaN,Admission Date: [**2151-7-16**] Dischar...



Last 1 in dataframe


,ROW_ID,SUBJECT_ID,HADM_ID,CHARTDATE,CHARTTIME,STORETIME,CATEGORY,DESCRIPTION,CGID,ISERROR,TEXT
2083179,2070661,31097,115637.0,2132-01-21,2132-01-21 18:05:00,2132-01-21 18:31:00,Nursing/other,Report,16023.0,NaN,NPN 1800\nNursing Addendum:\n[**Known lastname...


In [15]:
# Only need HADM_ID and TEXT. Save resources and trim dataframe
NOTES_DF_working = NOTES_DF[['HADM_ID', 'TEXT']]
DataUtils.exploreDataframe(NOTES_DF_working, showRecords=1)

dataframe shape: (2083180, 2)

dataframe info: 
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2083180 entries, 0 to 2083179
Data columns (total 2 columns):
 #   Column   Dtype  
---  ------   -----  
 0   HADM_ID  float64
 1   TEXT     object 
dtypes: float64(1), object(1)
memory usage: 31.8+ MB
None

Null value count by column:


HADM_ID    231836
TEXT            0
dtype: int64



First 1 in dataframe


,HADM_ID,TEXT
0,167853.0,Admission Date: [**2151-7-16**] Dischar...



Last 1 in dataframe


,HADM_ID,TEXT
2083179,115637.0,NPN 1800\nNursing Addendum:\n[**Known lastname...


In [16]:
# Drop Null values in relevant columns HADM_ID, TEXT
NOTES_DF_working = DataUtils.dropNullRows(dataFrame=NOTES_DF_working, columns='HADM_ID', showRecords=1)

Dropping rows where HADM_ID is/are null

Original dataFrame shape: (2083180, 2)
Original null value count by column:


HADM_ID    231836
TEXT            0
dtype: int64


*** Rows with nulls meeting criteria have been dropped

New values:
dataframe shape: (1851344, 2)

dataframe info: 
<class 'pandas.core.frame.DataFrame'>
Int64Index: 1851344 entries, 0 to 2083179
Data columns (total 2 columns):
 #   Column   Dtype  
---  ------   -----  
 0   HADM_ID  float64
 1   TEXT     object 
dtypes: float64(1), object(1)
memory usage: 42.4+ MB
None

Null value count by column:


HADM_ID    0
TEXT       0
dtype: int64



First 1 in dataframe


,HADM_ID,TEXT
0,167853.0,Admission Date: [**2151-7-16**] Dischar...



Last 1 in dataframe


,HADM_ID,TEXT
2083179,115637.0,NPN 1800\nNursing Addendum:\n[**Known lastname...


New dataframe returned


In [17]:
# Sanity check:
# 1) No nulls in HADM_ID
# 2) No nulls in TEXT

assert(NOTES_DF_working['HADM_ID'].isnull().sum()==0)
print(f'PASS: HADM_ID contains no null values')

assert(NOTES_DF_working['TEXT'].isnull().sum()==0)
print(f'PASS: TEXT contains no null values')



PASS: HADM_ID contains no null values
PASS: TEXT contains no null values



## Combine all notes for each HADM_ID

In [18]:
# Combine notes for each HADM_ID into one record. 
# Should now be only one record per HADM_ID (100% unique in showUniqueColVals)
NOTES_DF_combined = NOTES_DF_working.groupby('HADM_ID').agg({
                                             'TEXT': lambda x: ' '.join(x)
                                            })
NOTES_DF_combined.reset_index(inplace=True)

# Check uniqueness of HADM_ID. SHould be 100% unique and no nulls
DataUtils.showUniqueColVals(dataFrame=NOTES_DF_combined,
                            colName='HADM_ID')

Data type of column [HADM_ID] is: float64
Total number of rows: 58361
Unique values in column: 58361 [percent unique: 100.0%]
Null values in column: 0
List of unique values:
[100001. 100003. 100006. ... 199995. 199998. 199999.]

Top 5 records by frequency for HADM_ID
        HADM_ID  record_count
0      100001.0             1
38926  166803.0             1
38900  166757.0             1
38901  166758.0             1
38902  166760.0             1

Bottom 5 records by frequency for HADM_ID
     HADM_ID  record_count
0   100001.0             1
28  100040.0             1
27  100039.0             1
13  100021.0             1
2   100006.0             1


([100001.0, 166803.0, 166757.0, 166758.0, 166760.0],
 [100001.0, 100040.0, 100039.0, 100021.0, 100006.0])

In [19]:
# Explore and compare length of text column before and after merge
print(f'Explore TEXT length for NOTES_DF_working')
DataUtils.show_column_text_length_summary(NOTES_DF_working, 'TEXT')

print(f'')
print(f'')
print(f'Explore TEXT length for NOTES_DF_combined')
DataUtils.show_column_text_length_summary(NOTES_DF_combined, 'TEXT')


Explore TEXT length for NOTES_DF_working
Displaying length summary for column: TEXT
count    1851344
mean        1871
std         2612
min            3
25%          442
50%         1083
75%         1850
max        55728
Name: Length of text, dtype: object


Explore TEXT length for NOTES_DF_combined
Displaying length summary for column: TEXT
count      58361
mean       59383
std       133922
min           73
25%        15484
50%        26810
75%        54714
max      4776054
Name: Length of text, dtype: object


In [20]:
# Sanity check: NOTES_DF_combined

# 1) TEXT has no nulls
assert(NOTES_DF_combined['TEXT'].isnull().sum()==0)
print(f'PASS: TEXT contains no null values')

# 2) HADM_ID == 100% unique
assert(NOTES_DF_combined['HADM_ID'].nunique()==len(NOTES_DF_combined))
print(f'PASS: HADM_ID is 100% unique')

# 3) HADM_ID has no nulls
assert(NOTES_DF_combined['HADM_ID'].isnull().sum()==0)
print(f'PASS: HADM_ID contains no null values')



PASS: TEXT contains no null values
PASS: HADM_ID is 100% unique
PASS: HADM_ID contains no null values


# Merge datasets for two outputs
- Output 1: All notes remain separate but have the ICD9 code from SEQ_NO 1
- Output 2: Notes are combined together. One set of notes for each HADM_ID.

## Output 1: All notes remain separate but with the ICD9 code from SEQ_NO== 1

In [21]:
# Before merging tables we need to determine if there will be any record loss on an inner join
# Is there a 100% match between HADM_ID between the two tables

# Get a list of unique HADM_ID from NOTES_DF
NOTES_DF_UNIQUE_HADM_ID = NOTES_DF_working['HADM_ID'].unique()
print(f'Number of unique HADM_ID in NOTES: {len(NOTES_DF_UNIQUE_HADM_ID)}')

# Get a list of unique HADM_ID from 
DIAGNOSES_ICD_DF_working_UNIQUE_HADM_ID = DIAGNOSES_ICD_DF_working['HADM_ID'].unique()
print(f'Number of unique HADM_ID in DIAGNOSIS: {len(DIAGNOSES_ICD_DF_working_UNIQUE_HADM_ID)}')

# Find the intersection of the two lists for what HADM_ID will remain after join
COMMON_HADM_ID = list(set(NOTES_DF_UNIQUE_HADM_ID).intersection(DIAGNOSES_ICD_DF_working_UNIQUE_HADM_ID))
print(f'Number of valid HADM_ID to appear after join: {len(COMMON_HADM_ID)}')



Number of unique HADM_ID in NOTES: 58361
Number of unique HADM_ID in DIAGNOSIS: 58929
Number of valid HADM_ID to appear after join: 58328


In [22]:
# Inner join will address any missing HADM_ID values in the tables on either side of the join
# Data check (below) will verify merge was successful
mimic3_notes_separateDF = pd.merge(NOTES_DF_working, DIAGNOSES_ICD_DF_working, on='HADM_ID', how='inner')

# Show summary prior to persisting
DataUtils.exploreDataframe(mimic3_notes_separateDF, showRecords=2)  

dataframe shape: (1851243, 6)

dataframe info: 
<class 'pandas.core.frame.DataFrame'>
Int64Index: 1851243 entries, 0 to 1851242
Data columns (total 6 columns):
 #   Column      Dtype  
---  ------      -----  
 0   HADM_ID     float64
 1   TEXT        object 
 2   ROW_ID      int64  
 3   SUBJECT_ID  int64  
 4   SEQ_NUM     float64
 5   ICD9_CODE   object 
dtypes: float64(2), int64(2), object(2)
memory usage: 98.9+ MB
None

Null value count by column:


HADM_ID       0
TEXT          0
ROW_ID        0
SUBJECT_ID    0
SEQ_NUM       0
ICD9_CODE     0
dtype: int64



First 2 in dataframe


,HADM_ID,TEXT,ROW_ID,SUBJECT_ID,SEQ_NUM,ICD9_CODE
0,167853.0,Admission Date: [**2151-7-16**] Dischar...,250889,22532,1.0,01193
1,167853.0,Admission Date: [**2151-7-16**] Dischar...,250889,22532,1.0,01193



Last 2 in dataframe


,HADM_ID,TEXT,ROW_ID,SUBJECT_ID,SEQ_NUM,ICD9_CODE
1851241,175166.0,Neonatology\nBaby Girl [**Known lastname 1672*...,364188,31655,1.0,V3001
1851242,175166.0,NPN NICU\nTerm female admitted to NICU for sep...,364188,31655,1.0,V3001


In [23]:
_ = DataUtils.showUniqueColVals(mimic3_notes_separateDF,'SEQ_NUM')

Data type of column [SEQ_NUM] is: float64
Total number of rows: 1851243
Unique values in column: 1 [percent unique: 0.0%]
Null values in column: 0
List of unique values:
[1.]

Top 5 records by frequency for SEQ_NUM
   SEQ_NUM  record_count
0      1.0       1851243

Bottom 5 records by frequency for SEQ_NUM
   SEQ_NUM  record_count
0      1.0       1851243


In [24]:
# Sanity check: mimic3_notes_separateDF

# 1) Should have the same unique HADM_ID as identified by the set intersection above
# Check both ways. Result_1 should == Result_2
MERGED_UNIQUE_HADM_ID = mimic3_notes_separateDF['HADM_ID'].unique()
result_1 = list(set(MERGED_UNIQUE_HADM_ID) - set(COMMON_HADM_ID))
result_2 = list(set(COMMON_HADM_ID) - set(MERGED_UNIQUE_HADM_ID))

assert(len(result_1) == len(result_2)) 
print(f'PASS: New table has expected HADM_ID record set and length of: {len(MERGED_UNIQUE_HADM_ID)}')

# 2) Should have no nulls in TEXT
assert(mimic3_notes_separateDF['TEXT'].isnull().sum()==0)
print(f'PASS: TEXT contains no null values')

# 3) Should have no nulls in ICD9_CODE
assert(mimic3_notes_separateDF['ICD9_CODE'].isnull().sum()==0)
print(f'PASS: ICD9_CODE contains no null values')

# 4) SEQ_NUM==1 for all records
assert(mimic3_notes_separateDF['SEQ_NUM'].loc[mimic3_notes_separateDF['SEQ_NUM'].isin([1.0])].nunique())==1
print(f'PASS: SEQ_NUM is only 1.0 for all entries')





PASS: New table has expected HADM_ID record set and length of: 58328
PASS: TEXT contains no null values
PASS: ICD9_CODE contains no null values
PASS: SEQ_NUM is only 1.0 for all entries


In [25]:
%%time
# Persist the dataset
# Note that no ICD9 code filtering has not been completed at this point
mimic3_notes_separateDF.to_pickle(f'{DATA_DIR}/working/notes_separate.pkl.gz')

CPU times: user 2min 12s, sys: 311 ms, total: 2min 12s
Wall time: 2min 12s


## Output 2: Notes are combined together. One set of notes for each HADM_ID.

In [26]:
# Before merging tables we need to determine if there will be any record loss on an inner join
# Is there a 100% match between HADM_ID between the two tables?

# Get a list of unique HADM_ID from NOTES_DF
NOTES_DF_UNIQUE_HADM_ID = NOTES_DF_combined['HADM_ID'].unique()
print(f'Number of unique HADM_ID in NOTES: {len(NOTES_DF_UNIQUE_HADM_ID)}')

# Get a list of unique HADM_ID from 
DIAGNOSES_ICD_DF_working_UNIQUE_HADM_ID = DIAGNOSES_ICD_DF_working['HADM_ID'].unique()
print(f'Number of unique HADM_ID in DIAGNOSIS: {len(DIAGNOSES_ICD_DF_working_UNIQUE_HADM_ID)}')

# Find the intersection of the two lists for what HADM_ID will remain after join
COMMON_HADM_ID = list(set(NOTES_DF_UNIQUE_HADM_ID).intersection(DIAGNOSES_ICD_DF_working_UNIQUE_HADM_ID))
print(f'Number of valid HADM_ID to appear after join: {len(COMMON_HADM_ID)}')

Number of unique HADM_ID in NOTES: 58361
Number of unique HADM_ID in DIAGNOSIS: 58929
Number of valid HADM_ID to appear after join: 58328


In [27]:
mimic3_notes_combinedDF = pd.merge(NOTES_DF_combined, DIAGNOSES_ICD_DF_working, on='HADM_ID', how='inner')

# Show summary prior to persisting
DataUtils.exploreDataframe(mimic3_notes_combinedDF, showRecords=2)                                   

dataframe shape: (58328, 6)

dataframe info: 
<class 'pandas.core.frame.DataFrame'>
Int64Index: 58328 entries, 0 to 58327
Data columns (total 6 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   HADM_ID     58328 non-null  float64
 1   TEXT        58328 non-null  object 
 2   ROW_ID      58328 non-null  int64  
 3   SUBJECT_ID  58328 non-null  int64  
 4   SEQ_NUM     58328 non-null  float64
 5   ICD9_CODE   58328 non-null  object 
dtypes: float64(2), int64(2), object(2)
memory usage: 3.1+ MB
None

Null value count by column:


HADM_ID       0
TEXT          0
ROW_ID        0
SUBJECT_ID    0
SEQ_NUM       0
ICD9_CODE     0
dtype: int64



First 2 in dataframe


,HADM_ID,TEXT,ROW_ID,SUBJECT_ID,SEQ_NUM,ICD9_CODE
0,100001.0,Admission Date: [**2117-9-11**] ...,464704,58526,1.0,25013
1,100003.0,Admission Date: [**2150-4-17**] ...,446055,54610,1.0,53100



Last 2 in dataframe


,HADM_ID,TEXT,ROW_ID,SUBJECT_ID,SEQ_NUM,ICD9_CODE
58326,199998.0,Admission Date: [**2119-2-18**] ...,305582,27200,1.0,41401
58327,199999.0,Admission Date: [**2136-4-4**] D...,380657,40370,1.0,48284


In [28]:
# Sanity check: mimic3_notes_separateDF


# 1) Should have the same unique HADM_ID as identified by the set intersection above
# Check both ways. Result_1 should == Result_2
MERGED_UNIQUE_HADM_ID = mimic3_notes_combinedDF['HADM_ID'].unique()
result_1 = list(set(MERGED_UNIQUE_HADM_ID) - set(COMMON_HADM_ID))
result_2 = list(set(COMMON_HADM_ID) - set(MERGED_UNIQUE_HADM_ID))

assert(len(result_1) == len(result_2)) 
print(f'PASS: New table has expected HADM_ID record set')

# 2) Should have no nulls in TEXT
assert(mimic3_notes_combinedDF['TEXT'].isnull().sum()==0)
print(f'PASS: TEXT contains no null values')

# 3) Should have no nulls in ICD9_CODE
assert(mimic3_notes_combinedDF['ICD9_CODE'].isnull().sum()==0)
print(f'PASS: ICD9_CODE contains no null values')

# 4) SEQ_NUM==1 for all records
assert(mimic3_notes_combinedDF['SEQ_NUM'].loc[mimic3_notes_separateDF['SEQ_NUM'].isin([1.0])].nunique())==1
print(f'PASS: SEQ_NUM is only 1.0 for all entries')





PASS: New table has expected HADM_ID record set
PASS: TEXT contains no null values
PASS: ICD9_CODE contains no null values
PASS: SEQ_NUM is only 1.0 for all entries


In [29]:
%%time
# Persist the dataset
# Note that no ICD9 code filtering has not been completed at this point
mimic3_notes_combinedDF.to_pickle(f'{DATA_DIR}/working/notes_combined.pkl.gz')

CPU times: user 2min 3s, sys: 221 ms, total: 2min 3s
Wall time: 2min 3s


# Explore ICD9_CODE frequency

In [30]:
# Code frequency distribution for data frame with SEPARATE notes
topICD_codes, bottomICD_codesList = DataUtils.showUniqueColVals(mimic3_notes_separateDF, colName='ICD9_CODE', showRecords=10)

Data type of column [ICD9_CODE] is: object
Total number of rows: 1851243
Unique values in column: 2769 [percent unique: 0.1%]
Null values in column: 0
List of unique values:
['01193' '5191' '51884' ... 'V502' 'V292' '7765']

Top 10 records by frequency for ICD9_CODE
     ICD9_CODE  record_count
2732     V3001        157174
2735     V3101        109767
2731     V3000         87594
47        0389         76775
833      41401         64187
1159     51881         57269
822      41071         50747
2740     V3401         30819
911        431         28037
863       4241         25728

Bottom 10 records by frequency for ICD9_CODE
     ICD9_CODE  record_count
2513      9471             1
2221      8364             1
2413      8749             1
131       1430             1
235       1723             1
2478      9083             1
238       1726             1
299       1963             1
2594      9762             1
2595      9773             1


In [31]:
# Code frequency distribution for data frame with combined notes (all notes per HADM_ID are combined)
topICD_codes, bottomICD_codesList = DataUtils.showUniqueColVals(mimic3_notes_combinedDF, colName='ICD9_CODE', showRecords=10)

Data type of column [ICD9_CODE] is: object
Total number of rows: 58328
Unique values in column: 2769 [percent unique: 4.7%]
Null values in column: 0
List of unique values:
['25013' '53100' '49320' ... '6141' '6398' '88121']

Top 10 records by frequency for ICD9_CODE
     ICD9_CODE  record_count
833      41401          3497
2731     V3000          3427
2732     V3001          2695
47        0389          2043
822      41071          1747
863       4241          1139
1159     51881          1120
911        431          1007
2735     V3101           993
1096       486           721

Bottom 10 records by frequency for ICD9_CODE
     ICD9_CODE  record_count
2768     V7281             1
577      28411             1
598      28749             1
603      28802             1
605      28804             1
610      28981             1
611      28982             1
612      28989             1
616       2912             1
621      29289             1


# Custom stopword list

In [32]:
# Custom stopword list added here as part of experiment environment and support item generation process
custom_stop_words = ['discharge', 'diagnosis', 'medications', 'medication', 'disposition', 'condition', 'instructions', 
                     'status', 'secondary', 'changes', 'instruction', 'change', 'home', 'name', 'hospital', 'daily',
                     'hour', 'follow', 'care', 'time', 'day', 'week', 'with', 'disp', 'discharged', 'admitted', 
                     'namepattern', 'none', 'chief', 'complaint', 'physical', 'exam', 'present', 'illness', 'family',
                     'year', 'history','admission', 'social', 'medical', 'allergies']

# Added this to the original custom stopwords from above. Very frequent in corpus
custom_stop_words.extend(['date'])
print(f'Number of custom stopwords: {len(custom_stop_words)}')
print(f'Custom stopwords:')
print(custom_stop_words)


with open(f'{DATA_DIR}/working/custom_stop_words.pkl', 'wb') as f:
    pickle.dump(custom_stop_words, f)

# Retrieve stopwords with
#with open(f'{DATA_DIR}/working/custom_stop_words.pkl', 'rb') as f:
#   mynewlist = pickle.load(f)
          

Number of custom stopwords: 42
Custom stopwords:
['discharge', 'diagnosis', 'medications', 'medication', 'disposition', 'condition', 'instructions', 'status', 'secondary', 'changes', 'instruction', 'change', 'home', 'name', 'hospital', 'daily', 'hour', 'follow', 'care', 'time', 'day', 'week', 'with', 'disp', 'discharged', 'admitted', 'namepattern', 'none', 'chief', 'complaint', 'physical', 'exam', 'present', 'illness', 'family', 'year', 'history', 'admission', 'social', 'medical', 'allergies', 'date']


# Scratchpad

## Explore differences in order of ops (ICD9_CODE filter, SEQ_NO)

In [33]:
DataUtils.exploreDataframe(DIAGNOSES_ICD_DF)

dataframe shape: (651047, 5)

dataframe info: 
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 651047 entries, 0 to 651046
Data columns (total 5 columns):
 #   Column      Non-Null Count   Dtype  
---  ------      --------------   -----  
 0   ROW_ID      651047 non-null  int64  
 1   SUBJECT_ID  651047 non-null  int64  
 2   HADM_ID     651047 non-null  int64  
 3   SEQ_NUM     651000 non-null  float64
 4   ICD9_CODE   651000 non-null  object 
dtypes: float64(1), int64(3), object(1)
memory usage: 24.8+ MB
None

Null value count by column:


ROW_ID         0
SUBJECT_ID     0
HADM_ID        0
SEQ_NUM       47
ICD9_CODE     47
dtype: int64



First 5 in dataframe


,ROW_ID,SUBJECT_ID,HADM_ID,SEQ_NUM,ICD9_CODE
0,1297,109,172335,1.0,40301
1,1298,109,172335,2.0,486
2,1299,109,172335,3.0,58281
3,1300,109,172335,4.0,5855
4,1301,109,172335,5.0,4254



Last 5 in dataframe


,ROW_ID,SUBJECT_ID,HADM_ID,SEQ_NUM,ICD9_CODE
651042,639798,97503,188195,2.0,20280
651043,639799,97503,188195,3.0,V5869
651044,639800,97503,188195,4.0,V1279
651045,639801,97503,188195,5.0,5275
651046,639802,97503,188195,6.0,5569


In [34]:
# Original data prep added a filter on ICD9 Codes and then sorted based on "keep first" not SEQ=1.0

# Info from original data prep
## labels = ['4019', '5849', '51881', '53081']
## diagnosis_icd_filter = diagnosis_icd.loc[diagnosis_icd['ICD9_CODE'].isin(labels)]
## diagnosis_icd_filter_drop = diagnosis_icd_filter.sort_values('SEQ_NUM').drop_duplicates('HADM_ID', keep='first')


# Expected record results from original research file run
RECORDS_AFTER_FILTER = 43645
RECORDS_AFTER_KEEPFIRST = 32275

# ICD9_Codes used in research paper
labels = ['4019', '5849', '51881', '53081']

DIAGNOSIS_ICD_SANITY_orig_filter = DIAGNOSES_ICD_DF.loc[DIAGNOSES_ICD_DF['ICD9_CODE'].isin(labels)]
assert(RECORDS_AFTER_FILTER == len(DIAGNOSIS_ICD_SANITY_orig_filter))
print(f'PASS: Number of records after filter (step 1) match original research notbook')

DIAGNOSIS_ICD_SANITY_orig_sort = DIAGNOSIS_ICD_SANITY_orig_filter.sort_values('SEQ_NUM', ascending=True).drop_duplicates('HADM_ID', keep='first')
assert(RECORDS_AFTER_KEEPFIRST == len(DIAGNOSIS_ICD_SANITY_orig_sort))
print(f'PASS: Number of records after filter (step 1) then "keep first" (step 2) match original research notbook')

# Explore data changes and output
print(f'')
print(f'HADM_ID after filter then keep_first')
_, _ = DataUtils.showUniqueColVals(DIAGNOSIS_ICD_SANITY_orig_sort, 'HADM_ID', showRecords=10)

print(f'')
print(f'SEQ_NUM after filter then keep_first')
_, _ = DataUtils.showUniqueColVals(DIAGNOSIS_ICD_SANITY_orig_sort, 'SEQ_NUM', showRecords=10)




PASS: Number of records after filter (step 1) match original research notbook
PASS: Number of records after filter (step 1) then "keep first" (step 2) match original research notbook

HADM_ID after filter then keep_first
Data type of column [HADM_ID] is: int64
Total number of rows: 32275
Unique values in column: 32275 [percent unique: 100.0%]
Null values in column: 0
List of unique values:
[153714 174834 101463 ... 154989 129551 170349]

Top 10 records by frequency for HADM_ID
       HADM_ID  record_count
0       100001             1
21512   166937             1
21525   166975             1
21524   166973             1
21523   166969             1
21522   166962             1
21521   166960             1
21520   166956             1
21519   166955             1
21518   166953             1

Bottom 10 records by frequency for HADM_ID
    HADM_ID  record_count
0    100001             1
4    100009             1
5    100012             1
6    100016             1
7    100017             1

In [35]:
# Comparing order of operations. Original (above) did filter then sort to keep first records
# This one reverses and sorts to keep first record (primary diagnoses) then filter


# Original data prep added a filter on ICD9 Codes and then sorted based on "keep first" not SEQ=1.0
## labels = ['4019', '5849', '51881', '53081']
## diagnosis_icd_filter = diagnosis_icd.loc[diagnosis_icd['ICD9_CODE'].isin(labels)]
## diagnosis_icd_filter_drop = diagnosis_icd_filter.sort_values('SEQ_NUM').drop_duplicates('HADM_ID', keep='first')


# Expected record results
RECORDS_AFTER_FILTER = 43645
RECORDS_AFTER_KEEPFIRST = 32275

# ICD9_Codes used in research paper
labels = ['4019', '5849', '51881', '53081']


# Filter recordset down keeping only 'first' record based on SEQ_NUM over HADM_ID
DIAGNOSIS_ICD_SANITY_sort = DIAGNOSES_ICD_DF.sort_values('SEQ_NUM', ascending=True).drop_duplicates('HADM_ID', keep='first')

print(f'')
# Now keep only the ones with the right ICD9_CODE
DIAGNOSIS_ICD_SANITY_filter = DIAGNOSIS_ICD_SANITY_sort.loc[DIAGNOSIS_ICD_SANITY_sort['ICD9_CODE'].isin(labels)]


# Explore data changes and output
print(f'')
print(f'HADM_ID after keep_first then filter')
_, _ = DataUtils.showUniqueColVals(DIAGNOSIS_ICD_SANITY_filter, 'HADM_ID', showRecords=5)

print(f'')
print(f'SEQ_NUM after keep_first then filter')
_, _ = DataUtils.showUniqueColVals(DIAGNOSIS_ICD_SANITY_filter, 'SEQ_NUM', showRecords=5)





HADM_ID after keep_first then filter
Data type of column [HADM_ID] is: int64
Total number of rows: 1713
Unique values in column: 1713 [percent unique: 100.0%]
Null values in column: 0
List of unique values:
[106630 144320 188724 ... 127888 125857 134655]

Top 5 records by frequency for HADM_ID
      HADM_ID  record_count
0      100132             1
1151   167908             1
1149   167883             1
1148   167842             1
1147   167837             1

Bottom 5 records by frequency for HADM_ID
   HADM_ID  record_count
0   100132             1
5   100459             1
6   100460             1
7   100463             1
8   100524             1

SEQ_NUM after keep_first then filter
Data type of column [SEQ_NUM] is: float64
Total number of rows: 1713
Unique values in column: 1 [percent unique: 0.1%]
Null values in column: 0
List of unique values:
[1.]

Top 5 records by frequency for SEQ_NUM
   SEQ_NUM  record_count
0      1.0          1713

Bottom 5 records by frequency for SEQ_NUM